In [3]:
# Import libraries
from stgraph_trainer.datasets import load_province_temporal_data
from stgraph_trainer.datasets import data_diff
from stgraph_trainer.datasets import timeseries_to_supervised
from stgraph_trainer.datasets import preprocess_data_for_seq2seq
from stgraph_trainer.datasets import inverse_diff
from stgraph_trainer.utils import get_config_from_json
from stgraph_trainer.utils import compute_metrics
from stgraph_trainer.utils import save_predictions
from stgraph_trainer.utils import save_metrics
from stgraph_trainer.models import Encoder, Decoder, DecoderWithAttention
from stgraph_trainer.trainers import Seq2SeqTrainer
from stgraph_trainer.callbacks import PostPredictionCallback
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from functools import partial
from tensorflow.keras import Model

In [4]:
# Set up configs and parameters
data_config_file = os.getcwd() + '/configs/data_config.json'
seq2seq_config_file = os.getcwd() + '/configs/seq2seq_config.json'
model_name = 'seq2seq'
work_dir = os.getcwd() + '/results'

In [5]:
data_configs = get_config_from_json(data_config_file)
seq2seq_configs = get_config_from_json(seq2seq_config_file)

In [6]:
PROVINCES = data_configs['provinces']
SPLIT_DATE = data_configs['split_date']
TIME_STEPS = int(data_configs['time_steps'])
STATUS = data_configs['status']

In [8]:
ENC_DENSE_UNITS = int(seq2seq_configs['enc_dense_units'])
ENC_UNITS = int(seq2seq_configs['enc_units'])
DEC_UNITS = int(seq2seq_configs['dec_units'])
BATCH_SIZE = int(seq2seq_configs['batch_size'])
DROP_RATE = float(seq2seq_configs['drop_rate'])
RECURRENT_DROP_RATE = float(seq2seq_configs['recurrent_drop_rate'])
TRIALS = int(seq2seq_configs['trials'])
EPOCHS = int(seq2seq_configs['epochs'])

In [9]:
# Load and process dataset

df = pd.read_csv('../notebooks/stgraph_trainer/datasets/data/water_nitrite (1).csv', index_col=0)
df = df.set_index('dateTime')

#df = load_province_temporal_data(provinces=PROVINCES, status=STATUS)
#df = load_province_temporal_data(provinces=PROVINCES, status=STATUS)

X_train, y_train, X_test, y_test, _, raw_test, scaler = preprocess_data_for_seq2seq(df,
                                                                                    SPLIT_DATE,
                                                                                    TIME_STEPS)


In [13]:
PROVINCES = df.columns

In [14]:
PROVINCES.shape

(24,)

In [16]:
n_features = X_train.shape[-1]
n_test_samples = len(raw_test) - TIME_STEPS
output_size = X_train.shape[-1]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(6408, 7) (6408, 1) (2184, 7) (2184, 1)


In [17]:
# Functions to handle post-prediction
# including inverse transform and add back difference
# to make predictions back to original scale.
def inverse_transform(x, scaler, **kwargs):
  original_shape = x.shape
  return scaler.inverse_transform(x.reshape((-1, 1))).reshape(original_shape)
inv_trans = partial(inverse_transform, scaler=scaler)
inv_trans._order = 10

def inv_diff_1(x, raw_values):
  return x + raw_values
inv_diff = partial(inv_diff_1, raw_values=df.iloc[-(n_test_samples + 1):-1].values)
inv_diff._order = 20

tfms = [inv_trans, inv_diff]

In [18]:
# Create model, train and evaluate
rmse_results = []
mae_results = []

for trial in range(TRIALS):
  encoder = Encoder(ENC_DENSE_UNITS,
                    ENC_UNITS,
                    BATCH_SIZE,
                    dropout=DROP_RATE,
                    recurrent_dropout=RECURRENT_DROP_RATE)

  # decoder = Decoder(1,
  #                   DEC_UNITS,
  #                   BATCH_SIZE,
  #                   dropout=DROP_RATE,
  #                   recurrent_dropout=RECURRENT_DROP_RATE)

  decoder = DecoderWithAttention(1,
                                 DEC_UNITS,
                                 BATCH_SIZE,
                                 dropout=DROP_RATE,
                                 recurrent_dropout=RECURRENT_DROP_RATE)
  
  model = [encoder, decoder]

  train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)) \
                            .shuffle(10000) \
                            .batch(BATCH_SIZE, drop_remainder=True)
  test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE * 2)

  loss_func = tf.losses.MeanSquaredError()
  optimizer = tf.optimizers.Adam(learning_rate=0.01)

  trainer = Seq2SeqTrainer(model,
                           train_ds,
                           test_ds,
                           loss_func,
                           optimizer,
                           callbacks=[PostPredictionCallback(funcs=tfms)],
                           raw_test=df.iloc[-n_test_samples:].values)

  history = trainer.train(EPOCHS)
  print(history)

  predict = trainer.predict()
  save_predictions(predict,
                   model_name,
                   n_exp=trial,
                   columns=PROVINCES,
                   index=df.iloc[-n_test_samples:].index,
                   path=work_dir)
  print(predict.shape)

  m, m_avg = compute_metrics(df.iloc[-n_test_samples:], predict, metric='rmse')
  m = np.append(m, m_avg)
  rmse_results.append(m)
  print(m, m_avg)
  
  m, m_avg = compute_metrics(df.iloc[-n_test_samples:], predict, metric='mae')
  m = np.append(m, m_avg)
  mae_results.append(m)
  print(m, m_avg)

2024-04-26 14:31:38.122551: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-26 14:31:39.012307: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch: 1; Elapsed time: 15.27248501777649; Train loss: 0.990317; Test MSE: 0.244370; Test loss RMSE: 0.494338


2024-04-26 14:31:53.733598: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-26 14:31:54.612380: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch: 2; Elapsed time: 14.720093250274658; Train loss: 0.964666; Test MSE: 0.222963; Test loss RMSE: 0.472190


2024-04-26 14:32:09.349898: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-26 14:32:10.230054: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

2024-04-26 14:44:20.115040: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-26 14:44:20.983569: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch: 9; Elapsed time: 14.80473804473877; Train loss: 0.897904; Test MSE: 0.270466; Test loss RMSE: 0.520064


2024-04-26 14:44:35.507864: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-26 14:44:36.374757: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch: 10; Elapsed time: 14.523262023925781; Train loss: 0.867024; Test MSE: 0.371971; Test loss RMSE: 0.609895
{'epoch': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'train_loss': [0.9640213, 0.9571342, 0.95844764, 0.9250469, 0.9209707, 0.8965923, 0.8985773, 0.9010122, 0.897904, 0.86702424], 'test_loss': [0.22441468, 0.27688804, 0.23600501, 0.31924626, 0.2554238, 0.2641621, 0.24875641, 0.23045456, 0.27046606, 0.37197146], 'elapsed_time': [14.719954013824463, 14.637626886367798, 14.524487972259521, 14.564972162246704, 14.74314022064209, 14.91177487373352, 15.115317821502686, 14.610213994979858, 14.80473804473877, 14.523262023925781]}
(91, 24)
[0.49832389 0.75331666 0.59459796 0.49294256 0.4855075  0.42283211
 0.64106694 0.62189209 0.44778279 0.71050406 0.40824906 0.98998977
 0.51642306 0.48853525 0.5603007  0.65817125 0.57950097 0.54868588
 0.53985774 0.59172869 1.07350973 0.5428068  0.42745338 0.53845947
 0.60989464] 0.60989463879148
[0.32503857 0.48885568 0.37910907 0.34363541 0.30635798 0.27991

2024-04-26 14:44:37.247878: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


[array([0.3811764 , 0.64523068, 0.47294179, 0.35809691, 0.34019696,
        0.25423555, 0.50660504, 0.49567193, 0.33631173, 0.59814109,
        0.22604181, 0.94723069, 0.3312158 , 0.38386246, 0.42202052,
        0.51017935, 0.45987457, 0.42805406, 0.4049897 , 0.48743044,
        0.99526102, 0.4354469 , 0.27731915, 0.35855691, 0.49551487]),
 array([0.33079273, 0.61531933, 0.47955342, 0.33800946, 0.29661136,
        0.24310648, 0.4782123 , 0.48079044, 0.31785275, 0.57448214,
        0.19635663, 0.93554415, 0.30257059, 0.35354897, 0.42255934,
        0.49981785, 0.43385985, 0.4303792 , 0.36537578, 0.48347979,
        0.98975997, 0.41250491, 0.24292008, 0.32973817, 0.47815633]),
 array([0.43572151, 0.67745118, 0.52513028, 0.41555532, 0.36221032,
        0.32505301, 0.55057986, 0.55591286, 0.3900658 , 0.67820278,
        0.3013612 , 0.96328898, 0.42407756, 0.44835212, 0.50501956,
        0.59397814, 0.50860401, 0.52416861, 0.40894619, 0.54827055,
        1.01330125, 0.47106524, 0.33386318, 